<div align="center"><h1>Summer School --- HIP C/C++</h1></div>

---
## Prerequisites

To get the most out of this lab you should already be able to:

- Declare variables, write loops, and use if / else statements in C.
- Define and invoke functions in C.
- Allocate arrays in C.

No previous HIP knowledge is required.

---
## Objectives

By the time you complete this lab, you will be able to:

- Write, compile, and run C/C++ programs that both call CPU functions and **launch** GPU **kernels**.
- Control parallel **thread hierarchy** using **execution configuration**.
- Refactor serial loops to execute their iterations in parallel on a GPU.
- Allocate and free memory available to both CPUs and GPUs.
- Handle errors generated by HIP code.
- Accelerate CPU-only applications.

---
## Accelerated Systems

*Accelerated systems*, also referred to as *heterogeneous systems*, are those composed of both CPUs and GPUs. Accelerated systems run CPU programs which in turn, launch functions that will benefit from the massive parallelism providied by GPUs. This lab environment is an accelerated system which includes an AMD GPU. Information about this GPU can be queried with the `rocm-smi` (*Systems Management Interface*) command line command. Issue the `rocm-smi` command now, by `CTRL` + clicking on the code execution cell below. You will find these cells throughout this lab any time you need to execute code. The output from running the command will be printed just below the code execution cell after the code runs. After running the code execution block immediately below, take care to find and note the (VRAM, compute utilization, temperature, etc) of the GPU in the output.

In [1]:
!rocm-smi



========================================= ROCm System Management Interface =========================================
=================================================== Concise Info ===================================================
Device  [Model : Revision]    Temp    Power     Partitions      SCLK  MCLK     Fan  Perf  PwrCap       VRAM%  GPU%  
        Name (20 chars)       (Edge)  (Socket)  (Mem, Compute)                                                      
0       [0xb002 : 0xc1]       33.0°C  11.189W   N/A, N/A        None  2800Mhz  0%   auto  Unsupported    3%   1%    
        0x15bf                                                                                                      
=============================================== End of ROCm SMI Log ================================================


---
## Writing Application Code for the GPU

HIP is a C++ Runtime API and Kernel Language that allows developers to create portable applications for AMD and NVIDIA GPUs from single source code.

Below is a `.cpp` file. It contains two functions, the first which will run on the CPU, the second which will run on the GPU. Spend a little time identifying the differences between the functions, both in terms of how they are defined, and how they are invoked.

```cpp
void CPUFunction()
{
  printf("This function is defined to run on the CPU.\n");
}

__global__ void GPUFunction()
{
  printf("This function is defined to run on the GPU.\n");
}

int main()
{
  CPUFunction();

  GPUFunction<<<1, 1>>>();
  HIPDeviceSynchronize();
}
```

Here are some important lines of code to highlight, as well as some other common terms used in accelerated computing:

`__global__ void GPUFunction()`
  - The `__global__` keyword indicates that the following function will run on the GPU, and can be invoked **globally**, which in this context means either by the CPU, or, by the GPU.
  - Often, code executed on the CPU is referred to as **host** code, and code running on the GPU is referred to as **device** code.
  - Notice the return type `void`. It is required that functions defined with the `__global__` keyword return type `void`.

`GPUFunction<<<1, 1>>>();`
  - Typically, when calling a function to run on the GPU, we call this function a **kernel**.
  - When launching a kernel, we must provide an **execution configuration**, which is done by using the `<<< ... >>>` syntax just prior to passing the kernel any expected arguments.
  - At a high level, execution configuration allows programmers to specify the **thread hierarchy** for a kernel launch, which defines the number of thread groupings (called **blocks**), as well as how many **threads** to execute in each block. Execution configuration will be explored at great length later in the lab, but for the time being, notice the kernel is launching with `1` block of threads (the first execution configuration argument) which contains `1` thread (the second configuration argument).

`hipDeviceSynchronize();`
  - Unlike much C/C++ code, launching kernels is **asynchronous**: the CPU code will continue to execute *without waiting for the kernel launch to complete*.
  - A call to `HIPDeviceSynchronize`, a function provided by the HIP runtime, will cause the host (CPU) code to wait until the device (GPU) code completes, and only then resume execution on the CPU.

---
### Compiling and Running Accelerated HIP Code

The HIP platform ships with the [**ROCm HIP Compiler**](https://rocm.docs.amd.com/projects/HIPCC/en/latest/index.html) `hipcc`, which can compile HIP accelerated applications, both the host, and the device code they contain. For the purposes of this lab, `hipcc` discussion with be pragmatically scoped to suit our immediate needs. After completing the lab, For anyone interested in a deeper dive into `hipcc`, start with [the documentation](https://rocm.docs.amd.com/projects/HIPCC/en/latest/index.html).

`hipcc` will be very familiar to experienced `gcc` users. Compiling, for example, a `some-HIP.cpp` file, is simply:

`hipcc -o out some-HIP.cpp `
  - `hipcc` is the command line command for using the `hipcc` compiler.
  - `some-HIP.cpp` is passed as the file to compile.
  - The `-o` flag is used to specify the output file for the compiled program.

---
### Example01: Accelerate Vector Addition Application

The following challenge involves accelerating a CPU-only vector addition program, which, while not the most sophisticated program, will give you an opportunity to focus on what you have learned about GPU-accelerating an application with HIP.

[`01_vector_addition.cpp`](./examples/01_vector_addition/vector_addition.cpp) contains a functioning CPU-only vector addition application. You need to write a HIP kernel on the GPU and to do its work in parallel.

![vec_add_01](./images/vec_add_01.png)

![vec_add_02](./images/vec_add_02.png)


```cpp
/* --------------------------------------------------
Include lib
-------------------------------------------------- */
#include <stdio.h>
#include <math.h>
#include "hip/hip_runtime.h"
```

```cpp
/* --------------------------------------------------
vector addition kernel
-------------------------------------------------- */
__global__ void vector_addition(double *A, double *B, double *C, int n)
{
    int id = blockDim.x * blockIdx.x + threadIdx.x;
    if (id < n) C[id] = A[id] + B[id];
}
```

```cpp
/* --------------------------------------------------
Main program
-------------------------------------------------- */
int main(int argc, char *argv[]){

    /* Size of array */
    int N = 1024 * 1024;

    /* Bytes in array in double precision */
    size_t bytes = N * sizeof(double);

    /* Allocate memory for host arrays */
    double *h_A = (double*)malloc(bytes);
    double *h_B = (double*)malloc(bytes);
    double *h_C = (double*)malloc(bytes);

    /* Initialize host arrays */
    for(int i=0; i<N; i++){
        h_A[i] = sin(i) * sin(i); 
        h_B[i] = cos(i) * cos(i);
        h_C[i] = 0.0;
    }    

    /* Allocate memory for device arrays */
    double *d_A, *d_B, *d_C;
    hipMalloc(&d_A, bytes);
    hipMalloc(&d_B, bytes);
    hipMalloc(&d_C, bytes);

    /* Copy data from host arrays to device arrays */
    hipMemcpy(d_A, h_A, bytes, hipMemcpyHostToDevice);
    hipMemcpy(d_B, h_B, bytes, hipMemcpyHostToDevice);
    hipMemcpy(d_C, h_C, bytes, hipMemcpyHostToDevice);

    /* Set kernel configuration parameters
           thr_per_blk: number of threads per thread block
           blk_in_grid: number of thread blocks in grid */
    int thr_per_blk = 256;
    int blk_in_grid = ceil( float(N) / thr_per_blk );

    /* Launch vector addition kernel */
    vector_addition<<<blk_in_grid, thr_per_blk>>>(d_A, d_B, d_C, N);

    /* Copy data from device array to host array (only need result, d_C) */
    hipMemcpy(h_C, d_C, bytes, hipMemcpyDeviceToHost);

    /* Check for correct results */
    double sum       = 0.0;
    double tolerance = 1.0e-14;

    for(int i=0; i<N; i++){
        sum = sum + h_C[i];
    } 

    if( fabs( (sum / N) - 1.0 ) > tolerance ){
        printf("Error: Sum/N = %0.2f instead of ~1.0\n", sum / N);
        exit(1);
    }

    /* Free CPU memory */
    free(h_A);
    free(h_B);
    free(h_C);

    /* Free Device memory */
    hipFree(d_A);
    hipFree(d_B);
    hipFree(d_C);

    printf("\n==============================\n");
    printf("__SUCCESS__\n");
    printf("------------------------------\n");
    printf("N                : %d\n", N);
    printf("Blocks in Grid   : %d\n", blk_in_grid);
    printf("Threads per Block: %d\n", thr_per_blk);
    printf("==============================\n\n"); 

    return 0;
}
```

In [1]:
!mkdir -p build
!hipcc --offload-arch=gfx908 -Wno-unused-result -o build/vector_add_basic examples/01_vector_addition/vector_addition.cpp && build/vector_add_basic


__SUCCESS__
------------------------------
N                : 1048576
Blocks in Grid   : 4096
Threads per Block: 256



---
### Exercise01: Error check

Through the example, you have learned the basic operation mode of array addition. Now there is an error in the following part of the code. Please find it and modify it so that the program generates the correct answer. You can find the source code in [`01_error_check.cpp`](./exercises/01_error_check/vector_addition.cpp).


```cpp
#include <stdio.h>
#include <math.h>
#include "hip/hip_runtime.h"

/* Macro for checking GPU API return values */
#define gpuCheck(call)                                                                          \
do{                                                                                             \
    hipError_t gpuErr = call;                                                                   \
    if(hipSuccess != gpuErr){                                                                   \
        printf("GPU API Error - %s:%d: '%s'\n", __FILE__, __LINE__, hipGetErrorString(gpuErr)); \
        exit(1);                                                                                \
    }                                                                                           \
}while(0)

/* --------------------------------------------------
Vector addition kernel
-------------------------------------------------- */
__global__ void vector_addition(double *A, double *B, double *C, int n)
{
    int id = blockDim.x * blockIdx.x + threadIdx.x;
    if (id < n) C[id] = A[id] + B[id];
}

/* --------------------------------------------------
Main program
-------------------------------------------------- */
int main(int argc, char *argv[]){

    /* Size of array */
    int N = 1024 * 1024;

    /* Bytes in array in double precision */
    size_t bytes = N * sizeof(double);

    /* Allocate memory for host arrays */
    double *h_A = (double*)malloc(bytes);
    double *h_B = (double*)malloc(bytes);
    double *h_C = (double*)malloc(bytes);

    /* Initialize host arrays */
    for(int i=0; i<N; i++){
        h_A[i] = sin(i) * sin(i); 
        h_B[i] = cos(i) * cos(i);
        h_C[i] = 0.0;
    }    

    /* Allocate memory for device arrays */
    double *d_A, *d_B, *d_C;
    gpuCheck( hipMalloc(&d_A, bytes) );
    gpuCheck( hipMalloc(&d_B, bytes) );
    gpuCheck( hipMalloc(&d_C, bytes) );

    /* Copy data from host arrays to device arrays */
    gpuCheck( hipMemcpy(h_A, h_A, bytes, hipMemcpyHostToDevice) );
    gpuCheck( hipMemcpy(d_B, h_B, bytes, hipMemcpyHostToDevice) );
    gpuCheck( hipMemcpy(d_C, h_C, bytes, hipMemcpyHostToDevice) );

    /* Set kernel configuration parameters
           thr_per_blk: number of threads per thread block
           blk_in_grid: number of thread blocks in grid */
    int thr_per_blk = 256;
    int blk_in_grid = ceil( float(N) / thr_per_blk );

    /* Launch vector addition kernel */
    vector_addition<<<blk_in_grid, thr_per_blk>>>(d_A, d_B, d_C, N);

    /* Check for kernel launch errors */
    gpuCheck( hipGetLastError() );

    /* Check for kernel execution errors */
    gpuCheck ( hipDeviceSynchronize() );

    /* Copy data from device array to host array (only need result, d_C) */
    gpuCheck( hipMemcpy(h_C, d_C, bytes, hipMemcpyDeviceToHost) );

    /* Check for correct results */
    double sum       = 0.0;
    double tolerance = 1.0e-14;

    for(int i=0; i<N; i++){
        sum = sum + h_C[i];
    } 

    if( fabs( (sum / N) - 1.0 ) > tolerance ){
        printf("Error: Sum/N = %0.2f instead of ~1.0\n", sum / N);
        exit(1);
    }

    /* Free CPU memory */
    free(h_A);
    free(h_B);
    free(h_C);

    /* Free Device memory */
    gpuCheck( hipFree(d_A) );
    gpuCheck( hipFree(d_B) );
    gpuCheck( hipFree(d_C) );

    printf("\n==============================\n");
    printf("__SUCCESS__\n");
    printf("------------------------------\n");
    printf("N                : %d\n", N);
    printf("Blocks in Grid   : %d\n",  blk_in_grid);
    printf("Threads per Block: %d\n",  thr_per_blk);
    printf("==============================\n\n");

    return 0;
}
```

`Instructions:`

You can find the solution in ./exercises/01_error_check/solution/vector_addition.cpp

In [ ]:
# Run and test your program 
!hipcc --offload-arch=gfx908 -Wno-unused-result -o build/error_check exercises/01_error_check/vector_addition.cpp && build/error_check

---
### Exercise02: Increments each element of an array

In this exercise, you will be working with a simple HIP (Heterogeneous-Compute Interface for Portability) program that increments each element of an array by one using GPU acceleration. The program is mostly complete, but there is one part that you need to implement. You can find the source code in [`02_add_one.cpp`](./exercises/02_add_d2h_data_transfer/add_one.cpp).

`Background:`

The program performs the following steps:
1. Defines a kernel function add_one that adds one to each element of an integer array.
2. Allocates memory on the host (CPU) for an array of size N.
3. Initializes the host array elements to zero.
4. Allocates memory on the device (GPU) for an array of the same size.
5. Copies the data from the host array to the device array.
6. Configures the kernel launch parameters (number of threads per block and number of blocks in the grid).
7. Launches the kernel to execute on the device.
8. Copies the results back from the device array to the host array.
9. Verifies the results.
10. Frees the allocated memory on both the host and the device.


```cpp
#include <stdio.h>
#include <math.h>
#include "hip/hip_runtime.h"

/* Macro for checking GPU API return values */
#define gpuCheck(call)                                                                          \
do{                                                                                             \
    hipError_t gpuErr = call;                                                                   \
    if(hipSuccess != gpuErr){                                                                   \
        printf("GPU API Error - %s:%d: '%s'\n", __FILE__, __LINE__, hipGetErrorString(gpuErr)); \
        exit(1);                                                                                \
    }                                                                                           \
}while(0)

/* --------------------------------------------------
Add one kernel
-------------------------------------------------- */
__global__ void add_one(int *A, int n)
{
    int id = blockDim.x * blockIdx.x + threadIdx.x;
    if (id < n) A[id] = A[id] + 1;
}
```

`Task:`

Your task is to complete the part of the program that copies the data back from the device array to the host array. The function to use is `hipMemcpy`, which has the following definition:
```c
hipError_t hipMemcpy(void* destination_buffer,
                     void* source_buffer,
                     size_t num_bytes_to_copy,
                     hipMemcpyKind kind);
```
Refer to the existing `hipMemcpy` call used to copy data from the host to the device as a guide.

```cpp
/* --------------------------------------------------
Main program
-------------------------------------------------- */
int main(int argc, char *argv[]){

    /* Size of array */
    int N = 1024 * 1024;

    /* Bytes in N ints */
    size_t bytes = N * sizeof(int);

    /* Allocate memory for host array */
    int *h_A = (int*)malloc(bytes);

    /* Initialize host arrays */
    for(int i=0; i<N; i++){
        h_A[i] = 0; 
    }    

    /* Allocate memory for device array */
    int *d_A;
    gpuCheck( hipMalloc(&d_A, bytes) );

    /* Copy data from host array to device array */
    gpuCheck( hipMemcpy(d_A, h_A, bytes, hipMemcpyHostToDevice) );

    /* Set kernel configuration parameters
           thr_per_blk: number of threads per thread block
           blk_in_grid: number of thread blocks in grid */
    int thr_per_blk = 256;
    int blk_in_grid = ceil( float(N) / thr_per_blk );

    /* Launch kernel */
    add_one<<<blk_in_grid, thr_per_blk>>>(d_A, N);

    /* Check for kernel launch errors */
    gpuCheck( hipGetLastError() );

    /* Check for kernel execution errors */
    gpuCheck ( hipDeviceSynchronize() );

    /* Copy data from device array to host array */

    // /////////////////////////////////////////////////////////
    // TODO: Add hipMemcpy call here using the following
    //       definition.
    //
    //       hipError_t hipMemcpy( void*  destination_buffer,
    //                             void*  source_buffer,
    //                             size_t num_bytes_to_copy,
    //                             hipMemcpyKind kind
    //                           )
    // 
    //       If you get stuck, see the host-to-device call above
    // /////////////////////////////////////////////////////////

    /* Check for correct results */
    for (int i=0; i<N; i++){
        if(h_A[i] != 1){
            printf("Error: h_A[%d] = %d instead of 1\n", i, h_A[i]);
            exit(1);
        }
    }

    /* Free CPU memory */
    free(h_A);

    /* Free Device memory */
    gpuCheck( hipFree(d_A) );

    printf("\n==============================\n");
    printf("__SUCCESS__\n");
    printf("------------------------------\n");
    printf("N                : %d\n", N);
    printf("Blocks in Grid   : %d\n",  blk_in_grid);
    printf("Threads per Block: %d\n",  thr_per_blk);
    printf("==============================\n\n");

    return 0;
}
```

`Instructions:`

1. Locate the section in the code marked TODO.
2. Implement the hipMemcpy call to copy the data from the device array d_A back to the host array h_A.
3. Compile and run the program to ensure it executes correctly and prints __SUCCESS__ if all elements in the array have been incremented properly.
4. If any element is not correctly incremented, the program should print an error message specifying the index and incorrect value.

By completing this task, you will demonstrate your understanding of basic HIP memory management and kernel execution.

You can find the solution in ./exercises/02_add_d2h_data_transfer/solution/add_one.cpp

In [ ]:
# Run and test your program 
!hipcc --offload-arch=gfx908 -Wno-unused-result -o build/add_one exercises/02_add_d2h_data_transfer/add_one.cpp && build/add_one

---
### Exercise03: Squares each element of an array

In this exercise, you will be working with a HIP program that squares each element of an array using GPU acceleration. The program is mostly complete, but there is one part that you need to implement. You can find the source code in [`03_square.cpp`](./exercises/03_complete_square_elements/square_elements.cpp).

`Background:`

The program performs the following steps:

1. Defines a kernel function square_elements that squares each element of an integer array.
2. Allocates memory on the host (CPU) for an array of size N.
3. Initializes the host array elements to their index values.
4. Allocates memory on the device (GPU) for an array of the same size.
5. Copies the data from the host array to the device array.
6. Configures the kernel launch parameters (number of threads per block and number of blocks in the grid).
7. Launches the kernel to execute on the device.
8. Copies the results back from the device array to the host array.
9. Verifies the results.
10. Frees the allocated memory on both the host and the device.

```cpp
#include <stdio.h>
#include <math.h>
#include "hip/hip_runtime.h"

/* Macro for checking GPU API return values */
#define gpuCheck(call)                                                                          \
do{                                                                                             \
    hipError_t gpuErr = call;                                                                   \
    if(hipSuccess != gpuErr){                                                                   \
        printf("GPU API Error - %s:%d: '%s'\n", __FILE__, __LINE__, hipGetErrorString(gpuErr)); \
        exit(1);                                                                                \
    }                                                                                           \
}while(0)
```

`Task:`

Your task is to complete the kernel function that squares each element of the array. The kernel should square the element only if its index is within bounds.

```cpp
/* --------------------------------------------------
Square elements kernel
-------------------------------------------------- */
__global__ void square_elements(int *A, int n)
{
    int id = blockDim.x * blockIdx.x + threadIdx.x;


    // TODO: Complete the kernel by squaring
    //       all elements of the array.

}

/* --------------------------------------------------
Main program
-------------------------------------------------- */
int main(int argc, char *argv[]){

    /* Size of array */
    int N = 1024 * 1024;

    /* Bytes in N ints */
    size_t bytes = N * sizeof(int);

    /* Allocate memory for host array */
    int *h_A = (int*)malloc(bytes);

    /* Initialize host arrays */
    for(int i=0; i<N; i++){
        h_A[i] = i; 
    }    

    /* Allocate memory for device array */
    int *d_A;
    gpuCheck( hipMalloc(&d_A, bytes) );

    /* Copy data from host array to device array */
    gpuCheck( hipMemcpy(d_A, h_A, bytes, hipMemcpyHostToDevice) );

    /* Set kernel configuration parameters
           thr_per_blk: number of threads per thread block
           blk_in_grid: number of thread blocks in grid */
    int thr_per_blk = 256;
    int blk_in_grid = ceil( float(N) / thr_per_blk );

    /* Launch kernel */
    square_elements<<<blk_in_grid, thr_per_blk>>>(d_A, N);

    /* Check for kernel launch errors */
    gpuCheck( hipGetLastError() );

    /* Check for kernel execution errors */
    gpuCheck ( hipDeviceSynchronize() );

    /* Copy data from device array to host array */
    gpuCheck( hipMemcpy(h_A, d_A, bytes, hipMemcpyDeviceToHost) );

    /* Check for correct results */
    for (int i=0; i<N; i++){

        if(h_A[i] != i * i){
            printf("Error: h_A[%d] = %d instead of %d\n", i, h_A[i], i*i );
            exit(1);
        }
    }

    /* Free CPU memory */
    free(h_A);

    /* Free Device memory */
    gpuCheck( hipFree(d_A) );

    printf("\n==============================\n");
    printf("__SUCCESS__\n");
    printf("------------------------------\n");
    printf("N                : %d\n", N);
    printf("Blocks in Grid   : %d\n",  blk_in_grid);
    printf("Threads per Block: %d\n",  thr_per_blk);
    printf("==============================\n\n");

    return 0;
}
```

`Instructions:`

1. Locate the section in the kernel function square_elements marked TODO.
2. Implement the logic to square each element of the array.
3. Compile and run the program to ensure it executes correctly and prints __SUCCESS__ if all elements in the array have been squared properly.
4. If any element is not correctly squared, the program should print an error message specifying the index and incorrect value.

By completing this task, you will demonstrate your understanding of writing and executing basic HIP kernels for element-wise operations on arrays.

You can find the solution in ./exercises/03_complete_square_elements/solution/square_elements.cpp

In [ ]:
# Run and test your program 
!hipcc --offload-arch=gfx908 -Wno-unused-result -o build/squares exercises/03_complete_square_elements/square_elements.cpp && build/squares

---
### Exercise04: Multiply two square matrices

In this exercise, you will be working with a HIP program that multiplies two square matrices using GPU acceleration. The program is mostly complete, but there are a few parts that you need to implement. You can find the source code in [`04_multiply.cpp`](./exercises/04_complete_matrix_multiply/matrix_multiply.cpp).

`Background:`

The program performs the following steps:

1. Defines a kernel function matrix_multiply that multiplies two NxN matrices.
2. Allocates memory on the host (CPU) for three NxN matrices: A, B, and C.
3. Initializes the host matrices A and B with specific values and sets C to zero.
4. Allocates memory on the device (GPU) for the matrices.
5. Copies the data from the host matrices to the device matrices.
6. Configures the kernel launch parameters (number of threads per block and number of blocks in the grid).
7. Launches the kernel to execute on the device.
8. Copies the result back from the device matrix C to the host matrix.
9. Verifies the results.
10. Frees the allocated memory on both the host and the device.

```cpp
#include <stdio.h>
#include <math.h>
#include "hip/hip_runtime.h"

/* Macro for checking GPU API return values */
#define gpuCheck(call)                                                                          \
do{                                                                                             \
    hipError_t gpuErr = call;                                                                   \
    if(hipSuccess != gpuErr){                                                                   \
        printf("GPU API Error - %s:%d: '%s'\n", __FILE__, __LINE__, hipGetErrorString(gpuErr)); \
        exit(1);                                                                                \
    }                                                                                           \
}while(0)
```

`Task:`

Your task is to complete the kernel function that performs the matrix multiplication. Specifically, you need to:

1. Identify the correct elements of matrices A and B to multiply.

2. Store the result back into matrix C.

```cpp
/* --------------------------------------------------
Matrix multiply kernel
-------------------------------------------------- */
__global__ void matrix_multiply(double *A, double *B, double *C, int n)
{
    int col = blockDim.x * blockIdx.x + threadIdx.x;
    int row = blockDim.y * blockIdx.y + threadIdx.y;

    if (col < n && row < n){

        int index = n * row + col;
        double element = 0.0;

        for (int i=0; i<n; i++){

            int row_index = n * row + i;
            int col_index = n * i   + col;

            // TODO: Look at row_index and col_index above
            //       and determine which is the correct 
            //       element for A and which is the correct
            //       element for B below.
            element = element + A[??] * B[??]; 
        }

        // TODO: Copy the result back to the C array here
    }
}

/* --------------------------------------------------
Main program
-------------------------------------------------- */
int main(int argc, char *argv[]){

    /* Size of NxN matrix */
    int N = 1024;

    /* Bytes in matrix in double precision */
    size_t bytes = N * N * sizeof(double);

    double *h_A = (double*)malloc(bytes);
    double *h_B = (double*)malloc(bytes);
    double *h_C = (double*)malloc(bytes);

    /* Initialize host arrays */
    for(int i=0; i<N; i++){
        for(int j=0; j<N; j++){

            int index = N * i + j;

            h_A[index] = j + 1.0;
            h_B[index] = 1.0 / (i + 1.0);
            h_C[index] = 0.0;
        }
    }    

    /* Allocate memory for device matrices */
    double *d_A, *d_B, *d_C;
    gpuCheck( hipMalloc(&d_A, bytes) );
    gpuCheck( hipMalloc(&d_B, bytes) );
    gpuCheck( hipMalloc(&d_C, bytes) );

    /* Copy data from host matrices to device matricesL */
    gpuCheck( hipMemcpy(d_A, h_A, bytes, hipMemcpyHostToDevice) );
    gpuCheck( hipMemcpy(d_B, h_B, bytes, hipMemcpyHostToDevice) );
    gpuCheck( hipMemcpy(d_C, h_C, bytes, hipMemcpyHostToDevice) );

    /* Set kernel configuration parameters
           thr_per_blk: number of threads per thread block
           blk_in_grid: number of thread blocks in grid
    
       (NOTE: dim3 is a c struct with member variables x, y, z) */
    dim3 thr_per_blk( 16, 16, 1 );
    dim3 blk_in_grid( ceil( float(N) / thr_per_blk.x), ceil(float(N) / thr_per_blk.y), 1 );

    /* Launch matrix addition kernel */
    matrix_multiply<<<blk_in_grid, thr_per_blk>>>(d_A, d_B, d_C, N);

    /* Check for kernel launch errors */
    gpuCheck( hipGetLastError() );

    /* Check for kernel execution errors */
    gpuCheck ( hipDeviceSynchronize() );

    /* Copy data from device matrix to host matrix (only need result, d_C) */
    gpuCheck( hipMemcpy(h_C, d_C, bytes, hipMemcpyDeviceToHost) );

    /* Check for correct results */
    double tolerance = 1.0e-14;

    for(int i=0; i<N; i++){
        for(int j=0; j<N; j++){
                
            int index = N * i + j;
            if( isnan(h_C[index]) || fabs(h_C[index] - N ) > tolerance ){
                printf("Error: h_C[%d] = %0.14f instead of %d\n", index, h_C[index], N);
                exit(1);
            }
        }
    }   

    /* Free CPU memory */
    free(h_A);
    free(h_B);
    free(h_C);

    /* Free Device memory */
    gpuCheck( hipFree(d_A) );
    gpuCheck( hipFree(d_B) );
    gpuCheck( hipFree(d_C) );

    printf("\n==============================\n");
    printf("__SUCCESS__\n");
    printf("------------------------------\n");
    printf("N                  : %d\n", N);
    printf("X Blocks in Grid   : %d\n",  blk_in_grid.x);
    printf("X Threads per Block: %d\n",  thr_per_blk.x);
    printf("Y Blocks in Grid   : %d\n",  blk_in_grid.y);
    printf("Y Threads per Block: %d\n",  thr_per_blk.y);
    printf("==============================\n\n");

    return 0;
}
```

`Instructions:`

1. Locate the TODO comments in the kernel function matrix_multiply.
2. Determine the correct indices for accessing the elements of matrices A and B.
3. Write the code to store the computed value into matrix C.
4. Compile and run the program to ensure it executes correctly and prints __SUCCESS__ if all elements in the result matrix C are computed correctly.
5. If any element is not computed correctly, the program should print an error message specifying the index and incorrect value.

By completing this task, you will demonstrate your understanding of writing and executing basic HIP kernels for matrix operations.

You can find the solution in ./exercises/04_complete_matrix_multiply/solution/matrix_multiply.cpp

In [ ]:
# Run and test your program 
!hipcc --offload-arch=gfx908 -Wno-unused-result -o build/matrix_multiply exercises/04_complete_matrix_multiply/matrix_multiply.cpp && build/matrix_multiply

---
### Exercise05: Compare performance against the hipBLAS version of DGEMM

In this exercise, we will use the `matrix_multiply` kernel we completed in `04_complete_the_kernel` and compare its performance against the hipBLAS version of DGEMM. 

You will not need to make any code changes. Instead, you will simply compile the code and submit the job. This will run the code under the `rocprof` profiling tool and parse the results. 

To compile and run:
```
$ make

$ sbatch submit.sh
```

To view the resulting profile, run the python script:
```
./parse_output.py
```

It should be clear from the performance difference that using existing libraries is typically the right choice instead of re-inventing the (slower) wheel.

---
### Exercise06: Hipify the CUDA pingpong code

This code sends data back and forth between the host and device 50 times and calculates the bandwidth. 

Your job is to `hipify` the code, then compile and run it. 
NOTE: The `#include "hip/hip_runtime.h" doesn't always get added when a code is `hipify`-ed, so it might need to be added manually.

To compile and run:
```
$ make
 
$ sbatch submit.sh
```

Recall that the CPU and GPU are connected with PCIe4 (x16), which has a peak bandwidth of 32 GB/s. What percentage of the peak performance do we achieve?

---
### Exercise07: Multiply two square matrices using shared memory

In this exercise, you will be working with a HIP program that multiplies two square matrices using shared memory on the GPU for improved performance. The program is mostly complete, but there are a few parts that you need to implement. You can find the source code in [`07_multiply_shared.cpp`](./exercises/07_matrix_multiply_shared/matrix_multiply.cpp).

`Background:`

The program performs the following steps:

1. Defines a kernel function matrix_multiply that multiplies two NxN matrices using shared memory.
2. Allocates memory on the host (CPU) for three NxN matrices: A, B, and C.
3. Initializes the host matrices A and B with specific values and sets C to zero.
4. Allocates memory on the device (GPU) for the matrices.
5. Copies the data from the host matrices to the device matrices.
6. Configures the kernel launch parameters (number of threads per block and number of blocks in the grid).
7. Launches the kernel to execute on the device.
8. Copies the result back from the device matrix C to the host matrix.
9. Verifies the results.
10. Frees the allocated memory on both the host and the device.

```cpp
#include <stdio.h>
#include <math.h>
#include "hip/hip_runtime.h"

/* Macro for checking GPU API return values */
#define gpuCheck(call)                                                                          \
do{                                                                                             \
    hipError_t gpuErr = call;                                                                   \
    if(hipSuccess != gpuErr){                                                                   \
        printf("GPU API Error - %s:%d: '%s'\n", __FILE__, __LINE__, hipGetErrorString(gpuErr)); \
        exit(1);                                                                                \
    }                                                                                           \
}while(0)

#define THREADS_PER_BLOCK_X 16
#define THREADS_PER_BLOCK_Y 16
```

`Task:`

Your task is to complete the kernel function that performs the matrix multiplication using shared memory. Specifically, you need to:

1. Read data from global memory into shared memory.

2. Perform the matrix multiplication using shared memory.

3. Store the result back into the global memory.

```cpp
/* --------------------------------------------------
Matrix multiply kernel
-------------------------------------------------- */
__global__ void matrix_multiply(double *A, double *B, double *C, int n)
{
    __shared__ double s_A[THREADS_PER_BLOCK_Y][THREADS_PER_BLOCK_X];
    __shared__ double s_B[THREADS_PER_BLOCK_Y][THREADS_PER_BLOCK_X];

    int col  = blockDim.x * blockIdx.x + threadIdx.x;
    int row  = blockDim.y * blockIdx.y + threadIdx.y;

    int lcol = threadIdx.x;
    int lrow = threadIdx.y;

    int index  = n * row + col;

    if (col < n && row < n){

        int THREADS_PER_BLOCK = THREADS_PER_BLOCK_Y;
        int num_chunks        = n / THREADS_PER_BLOCK;

        double element = 0.0;

        for (int chunk=0; chunk<num_chunks; chunk++){ 

            // TODO: Read data from global GPU memory into shared memory

            __syncthreads();

            for (int i=0; i<THREADS_PER_BLOCK; i++){
                element = element + s_A[lrow][i] * s_B[i][lcol];
            }

            __syncthreads();
        }

        C[index] = element;
    }
}

/* --------------------------------------------------
Main program
-------------------------------------------------- */
int main(int argc, char *argv[]){

    /* Size of NxN matrix */
    int N = 1024;

    /* Bytes in matrix in double precision */
    size_t bytes = N * N * sizeof(double);

    double *h_A = (double*)malloc(bytes);
    double *h_B = (double*)malloc(bytes);
    double *h_C = (double*)malloc(bytes);

    /* Initialize host arrays */
    for(int i=0; i<N; i++){
        for(int j=0; j<N; j++){

            int index = N * i + j;

            h_A[index] = j + 1.0;
            h_B[index] = 1.0 / (i + 1.0);
            h_C[index] = 0.0;
        }
    }    

    /* Allocate memory for device matrices */
    double *d_A, *d_B, *d_C;
    gpuCheck( hipMalloc(&d_A, bytes) );
    gpuCheck( hipMalloc(&d_B, bytes) );
    gpuCheck( hipMalloc(&d_C, bytes) );

    /* Copy data from host matrices to device matricesL */
    gpuCheck( hipMemcpy(d_A, h_A, bytes, hipMemcpyHostToDevice) );
    gpuCheck( hipMemcpy(d_B, h_B, bytes, hipMemcpyHostToDevice) );
    gpuCheck( hipMemcpy(d_C, h_C, bytes, hipMemcpyHostToDevice) );

    /* Set kernel configuration parameters
           thr_per_blk: number of threads per thread block
           blk_in_grid: number of thread blocks in grid
    
       (NOTE: dim3 is a c struct with member variables x, y, z) */
    dim3 thr_per_blk( THREADS_PER_BLOCK_X, THREADS_PER_BLOCK_Y, 1 );
    dim3 blk_in_grid( ceil( float(N) / thr_per_blk.x), ceil(float(N) / thr_per_blk.y), 1 );

    /* Launch matrix addition kernel */
    matrix_multiply<<<blk_in_grid, thr_per_blk>>>(d_A, d_B, d_C, N);

    /* Check for kernel launch errors */
    gpuCheck( hipGetLastError() );

    /* Check for kernel execution errors */
    gpuCheck ( hipDeviceSynchronize() );

    /* Copy data from device matrix to host matrix (only need result, d_C) */
    gpuCheck( hipMemcpy(h_C, d_C, bytes, hipMemcpyDeviceToHost) );

    /* Check for correct results */
    double tolerance = 1.0e-14;

    for(int i=0; i<N; i++){
        for(int j=0; j<N; j++){
                
            int index = N * i + j;
            if( isnan(h_C[index]) || fabs(h_C[index] - N ) > tolerance ){
                printf("Error: h_C[%d] = %0.14f instead of %d\n", index, h_C[index], N);
                exit(1);
            }
        }
    }   

    /* Free CPU memory */
    free(h_A);
    free(h_B);
    free(h_C);

    /* Free Device memory */
    gpuCheck( hipFree(d_A) );
    gpuCheck( hipFree(d_B) );
    gpuCheck( hipFree(d_C) );

    printf("\n==============================\n");
    printf("__SUCCESS__\n");
    printf("------------------------------\n");
    printf("N                  : %d\n", N);
    printf("X Blocks in Grid   : %d\n",  blk_in_grid.x);
    printf("X Threads per Block: %d\n",  thr_per_blk.x);
    printf("Y Blocks in Grid   : %d\n",  blk_in_grid.y);
    printf("Y Threads per Block: %d\n",  thr_per_blk.y);
    printf("==============================\n\n");

    return 0;
}
```

`Instructions:`

1. Locate the TODO comments in the kernel function matrix_multiply.
2. Read data from global memory into the shared memory arrays s_A and s_B.
3. Write the code to store the computed value into matrix C.
4. Compile and run the program to ensure it executes correctly and prints __SUCCESS__ if all elements in the result matrix C are computed correctly.
5. If any element is not computed correctly, the program should print an error message specifying the index and incorrect value.

By completing this task, you will demonstrate your understanding of writing and executing basic HIP kernels for matrix operations using shared memory to optimize performance.

You can find the solution in ./exercises/07_matrix_multiply_shared/solution/matrix_multiply.cpp

In [ ]:
# Run and test your program 
!hipcc --offload-arch=gfx908 -Wno-unused-result -o build/matrix_multiply_shared exercises/07_matrix_multiply_shared/matrix_multiply.cpp && build/matrix_multiply_shared